## Extrct Features Environment

In [ ]:
%cd drive/MyDrive/Deep_Learning_Project/Vilio_reproduce

In [ ]:
!cd vilio/py-bottom-up-attention; pip3 install -r requirements.txt

In [ ]:
!pip3 install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

In [ ]:
!cd vilio/py-bottom-up-attention; python3 setup.py build develop

In [ ]:
!pwd

In [ ]:
!cp ../base-vg-labels/ep_107_1192087/pytorch_model.bin vilio/data/

In [ ]:
!cd vilio/py-bottom-up-attention; python3 detectron2_mscoco_proposal_maxnms.py --batchsize 4 --split img --weight vgattr --minboxes 50 --maxboxes 50 --dataroot ../hateful_memes/

## Set Up Environment

In [ ]:
%cd drive/MyDrive/Deep_Learning_Project/Vilio_reproduce/

In [ ]:
!git clone https://github.com/airsplay/py-bottom-up-attention.git

In [ ]:
!cd py-bottom-up-attention

In [ ]:
!pip3 install wikipedia

In [ ]:
!pip install PyDictionary

In [ ]:
!pip3 install -r py-bottom-up-attention/requirements.txt

In [ ]:
!pip3 install torch --upgrade

In [ ]:
!pip3 install --upgrade torch torchvision

In [ ]:
!python3 py-bottom-up-attention/setup.py build develop

In [ ]:
!python3 -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [ ]:
!pip3 install fvcore

## Generate Features

In [ ]:
!nvidia-smi

In [ ]:
!pip3 install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

In [ ]:
!pip3 install pyyaml==5.1
!pip3 install torch==1.9.0+cu102 torchvision==0.10.0+cu102 -f https://download.pytorch.org/whl/torch_stable.html
!pip3 install --upgrade pycocotools
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu102/torch1.9/index.html


In [ ]:
# check pytorch installation: 
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
assert torch.__version__.startswith("1.9")   # please manually install torch 1.9 if Colab changes its default version

In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog


## Combine Objects Description

In [ ]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
import os

# import some common libraries
import numpy as np
import cv2
import torch
import random

# import some common detectron2 utilities
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

from matplotlib import pyplot as plt

import io
import cv2
import numpy as np
from IPython.display import clear_output, Image, display
import PIL.Image

from ast import literal_eval
import pandas as pd
import wikipedia
from PyDictionary import PyDictionary


def showarray(a, fmt='jpeg'):
    a = np.uint8(np.clip(a, 0, 255))
    f = io.BytesIO()
    PIL.Image.fromarray(a).save(f, fmt)
    display(Image(data=f.getvalue()))

In [ ]:
print(os.getcwd())

/content/drive/My Drive/Deep_Learning_Project/Vilio_reproduce


In [ ]:
# Load VG Classes
data_path = '/content/drive/MyDrive/Deep_Learning_Project/Vilio_reproduce/py-bottom-up-attention/demo/data/genome/1600-400-20'

vg_classes = []
with open(os.path.join(data_path, 'objects_vocab.txt')) as f:
    for object in f.readlines():
        vg_classes.append(object.split(',')[0].lower().strip())

attr_classes = []
with open(os.path.join(data_path, 'attributes_vocab.txt')) as f:
    for object in f.readlines():
        attr_classes.append(object.split(',')[0].lower().strip())
        
MetadataCatalog.get("vg").thing_classes = vg_classes

In [ ]:
im = cv2.imread("/content/drive/MyDrive/Deep_Learning_Project/hateful_memes/img/73501.png")
im_rgb = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
showarray(im_rgb)

In [ ]:
# Load results

results = pd.read_csv('/content/drive/MyDrive/Deep_Learning_Project/hateful_memes/hm_vgattr3636_parsed_output.csv')

In [ ]:
results.head()

,Unnamed: 0,img_id,img_h,img_w,objects_id,objects_conf,attrs_id,attrs_conf,num_boxes,boxes,features
0,0,1829,800,638,[ 119 50 372 941 546 546 372 452 327 ...,[0.95626676 0.92466885 0.8605039 0.85878277 0...,[ 23 156 7 338 6 6 7 105 23 11 6 ...,[0.6389162 0.18531372 0.13934124 0.1577835 0...,36,[[192.9586 210.34647 379.8297 339.7587...,[[0.00000000e+00 0.00000000e+00 1.22361295e-01...
1,1,27150,364,550,[ 372 372 372 50 462 462 438 50 50 ...,[0.95784 0.9332068 0.6296269 0.50006616 0...,[ 0 0 0 14 11 11 4 14 14 56 11 14 11 11 14 ...,[0.29027286 0.25311744 0.16535084 0.45917663 0...,36,[[3.46339813e+02 7.87766876e+01 5.01577820e+02...,[[0. 0. 0. ... 0. ...
2,2,90842,800,391,[ 50 50 119 919 119 310 50 372 191 ...,[0.93861437 0.90108556 0.7651991 0.74301803 0...,[156 209 7 10 0 0 0 7 7 7 56 ...,[0.15323618 0.16666874 0.38815346 0.72948366 0...,36,[[138.90503 23.854029 354.8863 274.9088...,[[0.0000000e+00 1.5041822e-01 0.0000000e+00 .....
3,3,73501,512,512,[ 234 1496 50 327 372 546 198 467 274 ...,[0.91288483 0.81429434 0.79973793 0.746937 0...,[ 11 115 11 11 11 11 4 163 23 196 23 ...,[0.6803433 0.26876092 0.1887938 0.21447974 0...,36,[[1.2626323e+02 5.2939976e+01 3.4049152e+02 2....,[[4.1491752e-03 3.3804424e-02 4.1034531e-02 .....
4,4,63712,365,550,[ 191 50 1127 1127 50 440 234 1038 234 ...,[0.5866494 0.5391369 0.49425042 0.45221475 0...,[ 11 7 7 7 166 11 11 7 11 7 166 ...,[0.60890704 0.3482649 0.7540588 0.7632702 0...,36,[[3.31277954e+02 1.68662613e+02 3.85197876e+02...,[[9.4464086e-02 0.0000000e+00 0.0000000e+00 .....


In [ ]:
# try_1235 = results.loc[results['img_id'] == 73501,['objects_id']]
# list_str = try_1235['objects_id'][3].replace('\n','').replace('[','').replace(']','').split(' ')
# list_str = list(filter(None, list_str))
# for i in list_str:
#   print(MetadataCatalog.get("vg").thing_classes[int(i)])

In [ ]:
# Get Wikipedia result
dictionary=PyDictionary()

metadata = MetadataCatalog.get("vg").thing_classes
wiki_pair = dict()

In [ ]:
for element in metadata:
  dic_m = dictionary.meaning(element.lower())
  if dic_m is not None:
    wiki_pair[element] = dic_m[list(dic_m.keys())[0]][0]
  else: 
    try:
      sumary = wikipedia.summary(wikipedia.search(element)[0])
      wiki_pair[element] = sumary
    except wikipedia.DisambiguationError as e:
      wiki_pair[element] = 'Ambiguation'
    except:
      wiki_pair[element] = 'No Page'
  

In [ ]:
attr_pair = dict()
for att in attr_classes:
  dic_m = dictionary.meaning(att.lower())
  if (dic_m is not None):
    if len(dic_m[list(dic_m.keys())[0]]) > 0:
      attr_pair[att] = dic_m[list(dic_m.keys())[0]][0]
    elif (len(dic_m.keys()) == 1) & (len(dic_m[list(dic_m.keys())[0]]) == 0):
      attr_pair[att] = 'No Page'
    elif (len(dic_m.keys()) > 1) & (len(dic_m[list(dic_m.keys())[0]]) == 0):
      attr_pair[att] = dic_m[list(dic_m.keys())[1]][0]
    else:
      attr_pair[att] = 'No Page'
  else: 
    try:
      sumary = wikipedia.summary(wikipedia.search(att)[0])
      attr_pair[att] = sumary
    except wikipedia.DisambiguationError as e:
      attr_pair[att] = 'Ambiguation'
    except:
      attr_pair[att] = 'No Page'

In [ ]:
# c = 0
# for k, v in wiki_pair.items():  # for name, age in dictionary.iteritems():  (for Python 2.x)
#   if v == 'No Page':
#     c += 1
#     print(k)
#   elif v == 'Ambiguation':
#     c += 1
#     print(k)
#   elif v is None:
#     c += 1
#     print(k)

Mapping the description to the major Object of image. Each image pick the top two objects, and the objects is based on the size of the box and the probabilities. 

In [ ]:
def convert_to_list(str_input):
  list_str = str_input.replace('\n','').replace('[','').replace(']','').split(' ')
  list_str = list(filter(None, list_str))
  return list_str

In [ ]:
def list_over_list(str_input):
  list_str = str_input.replace('\n','').replace('      ',' ').replace('     ',' ').replace('    ',' ').replace('   ',' ').replace('  ',' ').replace(' ]',']').replace('[ ','[').replace(' ',',')
  list_str = literal_eval(list_str)
  return list_str


In [ ]:
def area(list_list):
  out_list = []
  for l in list_list:
    out_list.append(l[2] * l[3])
  return out_list

In [ ]:
def obj_score(obj_conf, obj_area):
  w_conf = 0.8
  w_area = 0.5
  final_score = w_conf * np.array(obj_conf).astype('float32') + w_area * np.array(obj_area).astype('float32')
  return final_score

In [ ]:
def select_obj(score, objs):
  index = np.argmax(score)
  final_obj = objs[index]
  return final_obj

In [ ]:
def combine_str(str1, str2):
  if str1 in ['Ambiguation','No Page']:
    return str2
  elif str2 in ['Ambiguation','No Page']:
    return str1
  else:
    return str1.strip() + '|' + str2.strip()

In [ ]:
ready_results = results.copy()
ready_results.loc[:,'objects_id'] = ready_results.apply(lambda x: convert_to_list(x['objects_id']), 1)
ready_results.loc[:,'objects_conf'] = ready_results.apply(lambda x: convert_to_list(x['objects_conf']), 1)
ready_results.loc[:,'attrs_id'] = ready_results.apply(lambda x: convert_to_list(x['attrs_id']), 1)
ready_results.loc[:,'attrs_conf'] = ready_results.apply(lambda x: convert_to_list(x['attrs_conf']), 1)
ready_results.loc[:,'boxes'] = ready_results.apply(lambda x: list_over_list(x['boxes']), 1)


In [ ]:
ready_results.loc[:,'area_obj'] = ready_results.apply(lambda x: area(x['boxes']), 1)
ready_results.loc[:,'area'] = ready_results.apply(lambda x: x['img_h'] * x['img_w'], 1)
ready_results.loc[:,'area_obj_perc'] = ready_results.apply(lambda x: np.array(x['area_obj'])/x['area'], 1)
ready_results.loc[:,'obj_score'] = ready_results.apply(lambda x: obj_score(x['objects_conf'], x['area_obj_perc']), 1)
ready_results.loc[:,'select_obj'] = ready_results.apply(lambda x: select_obj(x['obj_score'], x['objects_id']), 1)
ready_results.loc[:,'obj_attr'] = ready_results.apply(lambda x: select_obj(x['obj_score'], x['attrs_id']), 1)

In [ ]:
# Get the Obj and correspoinding desc
ready_results.loc[:,'obj_nm'] = ready_results.apply(lambda x: metadata[int(x['select_obj'])] , 1)
ready_results.loc[:,'attri_nm'] = ready_results.apply(lambda x: attr_classes[int(x['obj_attr'])],1)
ready_results.loc[:,'obj_desc'] = ready_results.apply(lambda x: wiki_pair[x['obj_nm']] , 1)
ready_results.loc[:,'attri_desc'] = ready_results.apply(lambda x: attr_pair[x['attri_nm']] , 1)

In [ ]:
ready_results.loc[:,'desc'] = ready_results.apply(lambda x: combine_str(x['obj_desc'], x['attri_desc']), 1)

Map the Obj Description to the Images.

In [ ]:
ready_results.head()

,Unnamed: 0,img_id,img_h,img_w,objects_id,objects_conf,attrs_id,attrs_conf,num_boxes,boxes,features,area_obj,area,area_obj_perc,obj_score,select_obj,obj_attr,obj_nm,attri_nm,obj_desc,attri_desc,desc
0,0,1829,800,638,"[119, 50, 372, 941, 546, 546, 372, 452, 327, 5...","[0.95626676, 0.92466885, 0.8605039, 0.85878277...","[23, 156, 7, 338, 6, 6, 7, 105, 23, 11, 6, 23,...","[0.6389162, 0.18531372, 0.13934124, 0.1577835,...",36,"[[192.9586, 210.34647, 379.8297, 339.75873], [...",[[0.00000000e+00 0.00000000e+00 1.22361295e-01...,"[129050.45648828101, 451149.81291404, 384306.0...",510400,"[0.2528418034645004, 0.8839142102547806, 0.752...","[0.8914343, 1.1816921, 1.0648785, 1.113147, 0....",50,156,man,smiling,an adult person who is male (as opposed to a w...,a facial expression characterized by turning u...,an adult person who is male (as opposed to a w...
1,1,27150,364,550,"[372, 372, 372, 50, 462, 462, 438, 50, 50, 162...","[0.95784, 0.9332068, 0.6296269, 0.50006616, 0....","[0, 0, 0, 14, 11, 11, 4, 14, 14, 56, 11, 14, 1...","[0.29027286, 0.25311744, 0.16535084, 0.4591766...",36,"[[346.339813, 78.7766876, 501.57782, 248.48222...",[[0. 0. 0. ... 0. ...,"[124633.17222267168, 50958.27767393273, 62317....",200200,"[0.6225433177955628, 0.25453685151814553, 0.31...","[1.0775437, 0.8738339, 0.6593401, 0.75378466, ...",372,0,hand,gray,the (prehensile,a neutral achromatic color midway between whit...,the (prehensile|a neutral achromatic color mid...
2,2,90842,800,391,"[50, 50, 119, 919, 119, 310, 50, 372, 191, 50,...","[0.93861437, 0.90108556, 0.7651991, 0.74301803...","[156, 209, 7, 10, 0, 0, 0, 7, 7, 7, 56, 7, 7, ...","[0.15323618, 0.16666874, 0.38815346, 0.7294836...",36,"[[138.90503, 23.854029, 354.8863, 274.9088], [...",[[0.0000000e+00 1.5041822e-01 0.0000000e+00 .....,"[97561.36686944, 249371.69252294, 148731.67201...",312800,"[0.31189695290741687, 0.7972240809556905, 0.47...","[0.90683997, 1.1194805, 0.84990174, 0.6625915,...",50,209,man,standing,an adult person who is male (as opposed to a w...,social or financial or professional status or ...,an adult person who is male (as opposed to a w...
3,3,73501,512,512,"[234, 1496, 50, 327, 372, 546, 198, 467, 274, ...","[0.91288483, 0.81429434, 0.79973793, 0.746937,...","[11, 115, 11, 11, 11, 11, 4, 163, 23, 196, 23,...","[0.6803433, 0.26876092, 0.1887938, 0.21447974,...",36,"[[126.26323, 52.939976, 340.49152, 206.95631],...",[[4.1491752e-03 3.3804424e-02 4.1034531e-02 .....,"[70466.8685654912, 218265.3812813435, 199713.0...",262144,"[0.26880977083393554, 0.8326163531545391, 0.76...","[0.8647128, 1.0677437, 1.0207126, 0.7287651, 0...",1496,115,sleeve,long,the part of a garment that is attached at the ...,desire strongly or persistently,the part of a garment that is attached at the ...
4,4,63712,365,550,"[191, 50, 1127, 1127, 50, 440, 234, 1038, 234,...","[0.5866494, 0.5391369, 0.49425042, 0.45221475,...","[11, 7, 7, 7, 166, 11, 11, 7, 11, 7, 166, 11, ...","[0.60890704, 0.3482649, 0.7540588, 0.7632702, ...",36,"[[331.277954, 168.662613, 385.197876, 244.9898...",[[9.4464086e-02 0.0000000e+00 0.0000000e+00 .....,"[94369.56523918408, 178096.23984245965, 103734...",200750,"[0.47008500741810255, 0.8871543703235848, 0.51...","[0.70436203, 0.8748867, 0.6537668, 0.53657186,...",50,7,man,white,an adult person who is male (as opposed to a w...,a Caucasian,an adult person who is male (as opposed to a w...


In [ ]:
# for the disctionary between image and obj and attributes
final_out = ready_results.loc[:,['img_id','desc']]
img_desc_dict = final_out.to_dict()

## Convert the Description to TSV format

In [ ]:
!pwd

In [ ]:
# Combine the Description with the JSON sentences
import json

raw_data = []
splits = ['train', 'traindev','test_seen','test_unseen','dev_seen','dev_unseen'] # , 'traindev','test_seen','test_unseen','dev_seen','dev_unseen'
for split in splits:
  with open(os.path.join("/content/drive/MyDrive/Deep_Learning_Project/hateful_memes/", f"{split}.jsonl"), 'r') as j:
    for line in j:
      raw_data.append(json.loads(line))

In [ ]:
len(raw_data)

In [ ]:
df_train.head()

In [ ]:
df_train = pd.DataFrame.from_records(raw_data)
df_train.loc[:,'id'] = df_train.apply(lambda x:int(x['id']),1)

In [ ]:
cb_train = pd.merge(left = df_train, right = final_out, left_on = 'id', right_on = 'img_id')
cb_train.loc[:,'text'] = cb_train.apply(lambda x: combine_str(x['text'], x['desc']), 1)

In [ ]:
cb_train

,id,img,label,text,img_id,desc
0,42953,img/42953.png,0,its their character not their color that matte...,42953,an adult person who is male (as opposed to a w...
1,23058,img/23058.png,0,don't be afraid to love again everyone is not ...,23058,an adult person who is male (as opposed to a w...
2,13894,img/13894.png,0,putting bows on your pet|feline mammal usually...,13894,feline mammal usually having thick soft fur an...
3,37408,img/37408.png,0,i love everything and everybody! except for sq...,37408,the sense organ for hearing and equilibrium|a ...
4,82403,img/82403.png,0,"everybody loves chocolate chip cookies, even h...",82403,an adult person who is male (as opposed to a w...
...,...,...,...,...,...,...
8495,10423,img/10423.png,1,nobody wants to hang auschwitz me|an adult per...,10423,an adult person who is male (as opposed to a w...
8496,98203,img/98203.png,1,when god grants you a child after 20 years of ...,98203,an adult person who is male (as opposed to a w...
8497,36947,img/36947.png,1,gays on social media: equality! body positivit...,36947,an adult female person (as opposed to a man|a ...
8498,16492,img/16492.png,1,having a bad day? you could be a siamese twin ...,16492,an adult person who is male (as opposed to a w...


In [ ]:
# Re_preduce JSONL file
train_out = cb_train.loc[:,['id','img','label','text']].to_json(orient='records', lines=True)

In [ ]:
string = '{"img":"img/42953.png"}'
json.dumps(json.loads(string))

'{"img": "img/42953.png"}'

In [ ]:
c = 0
train_list = train_out.split('\n')
for c in range(20): # i in train_out.split('\n'):
  print(json.dumps(json.loads(train_list[c])))
  

In [ ]:
with open('/content/drive/MyDrive/Deep_Learning_Project/hateful_memes/train_cb.jsonl', 'w') as outfile:
    for entry in train_out.split('\n'):
        outfile.write(json.dumps(json.loads(entry)))
        outfile.write('\n')

In [ ]:
raw_data1 = []
splits = 'train_cb' #, 'traindev','test_seen','test_unseen'
with open(os.path.join("/content/drive/MyDrive/Deep_Learning_Project/hateful_memes/", f"{splits}.jsonl"), 'r') as j:
  for line in j:
    raw_data1.append(json.loads(line))

In [ ]:
raw_data1

In [ ]:
raw_data